# Final Project
For my final project, I wanted to make a program that reads a CSV file and make a graph out of the file. After creating the graph, the program will find the shortest path between a chosen vertex and all other vertices in the graph. To do this, the program implements two algorithms, Dijstra's algorithm and Bellman-Ford Algorithm.

These are the packages that I used to implement these two algoritms:

In [1]:
using Pkg

Pkg.add("Test")
Pkg.add("Graphs")
Pkg.add("MetaGraphs")
Pkg.add("TikzGraphs")
Pkg.add("CSV")
Pkg.add("SimpleWeightedGraphs")

   Resolving package versions...


  No Changes to `~/.julia/environment/v1.8/Project.toml`
  No Changes to `~/.julia/environment/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environment/v1.8/Project.toml`
  No Changes to `~/.julia/environment/v1.8/Manifest.toml`


   Resolving package versions...


...

  No Changes to `~/.julia/environment/v1.8/Project.toml`
  No Changes to `~/.julia/environment/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environment/v1.8/Project.toml`
  No Changes to `~/.julia/environment/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environment/v1.8/Project.toml`
  No Changes to `~/.julia/environment/v1.8/Manifest.toml`


In [2]:
using Test
using Graphs
using MetaGraphs
using TikzGraphs
using CSV
using SimpleWeightedGraphs

## Goal of Lecture
Introduction to shortest paths in graphs. In this lecture, we are going to discuss Dijkstra's Algorithm and Bellman-Ford Algorithm for finding shortest paths. We will go over each application's intuition, applications,  and it's limitations.

# Read CSV file

- The first thing that needs to be done is to read the CSV file to find all vertices and edges.
- Each line of the CSV file is formatted as "u,v,10", this represents an edge between a vertex u and vertex v with weight 10.
- The function read_csv() reads the CSV file and returns an array of tuples that contains the name of the two vertices and the weight between the two vertices

In [3]:
## Input: A CSV file
## Output: An array of tuples
## The function reads the csv file passed in and converts each line into a tuple for graph creation
function read_csv(csv_file)
    ## Reads CSV file
    file = CSV.File(csv_file)
    val_arr = []
    ## Loop through each row of CSV file and create a tuple to push into val_arr
    for row in file
        push!(val_arr,(string(row.Start),string(row.End),row.Weight))
    end
    return val_arr
end

read_csv (generic function with 1 method)

# Creating Graph
- The function create_graph() takes the array of tuples made in read_csv() and detects if there are negative weights in the graph. If there are negative weights, the algorithm will use the Bellman-Ford algorithm, otherwise it will use Dijkstra's algorithm.
- It should be noted that Bellman-Ford algorithm only works on a directed graph. This will be explained later...
- Another note, although this algorithm uses an undirected graph for Dijkstra's algorithm, the algorithm also works the same on a directed graph, it just needs to follow the directed edges

In [4]:
## Input: An array of tuples
## Output: A directed graph or an undirected graph
## The function looks for negative weight edges and determines whether to make a directed graph or undirected graph
function create_graph(tup_arr)
    is_neg = false
    ## Loops through all the weights and looks for a negative value
    for i in tup_arr
        if i[3] < 0
            is_neg = true
        end
    end
    if is_neg == false
        ## Since there are no negative weights, create an undirected graph
        println("Using Dijsktra's Algorithm")
        return create_undirected_graph(tup_arr)
    else
        ## Since there are negative weights, create a directed graph
        println("Using Bellman-Ford's Algorithm")
        return create_directed_graph(tup_arr)
    end
end

create_graph (generic function with 1 method)

In [5]:
## Input: An array of tuples
## Output: An undirected graph
## The function reads each tuple, determines the number of vertices and assigns edges (with weights) between the vertices. This function uses a MetaGraph to
## assign names to vertices and weights to the edges
function create_undirected_graph(tup_arr)
    ## Find all unique vertices in array
    unique_arr = []
    for i in tup_arr
        if (i[1] in unique_arr) == false
            push!(unique_arr,i[1])
        end
        if (i[2] in unique_arr) == false
            push!(unique_arr,i[2])
        end
    end
    ## Create Metagraph with the number of unique vertices found in the CSV file
    mg = MetaGraph(SimpleGraph(size(unique_arr,1)))
    count = 1
    ## Loops through each of the vertices and assigns the name given in the CSV file. Also set indexing property for easier access to vertices
    for j in vertices(mg)
        set_prop!(mg,j,:name,unique_arr[count])
        count += 1
    end
    set_indexing_prop!(mg,:name)
    ## Loop through the array of tuples and assign edges between the vertices with the given weight
    for k in tup_arr
        add_edge!(mg,mg[k[1],:name],mg[k[2],:name])
        set_prop!(mg,Edge(mg[k[1],:name],mg[k[2],:name]),:weight,k[3])
    end
    ## Return Metagraph
    return mg
end

create_undirected_graph (generic function with 1 method)

In [6]:
## Input: An array of tuples
## Output: An directed graph
## The function reads each tuple, determines the number of vertices and assigns edges (with weights) between the vertices. This function uses the
## SimpleWeightedGraphs package to create a directed graph with weights. Since MetaGraph couldn't be used to assign names to each vertex, a dictionary
## is used instead where the name of the vertex is the key for the index of the vertex
function create_directed_graph(tup_arr)
    ## Find all unique vertices in array
    unique_arr = []
    for i in tup_arr
        if (i[1] in unique_arr) == false
            push!(unique_arr,i[1])
        end
        if (i[2] in unique_arr) == false
            push!(unique_arr,i[2])
        end
    end
    ## Create a simple weighted directed graph
    g = SimpleWeightedDiGraph(size(unique_arr,1))
    ## Create dictionary where name of vertex is associated with index of that vertex
    dict_key = Dict()
    for j in 1:size(unique_arr,1)
        dict_key[unique_arr[j]] = j
    end
    ## Loop through array of tuples and assign edges between the vertices with the given weight
    for k in tup_arr
        add_edge!(g,dict_key[k[1]],dict_key[k[2]],k[3])
    end
    ## Returns the graph and dictionary
    return (g,dict_key)
end

create_directed_graph (generic function with 1 method)

Testing to see if the graph created from the CSV file is correct:

In [7]:
arr = read_csv("DijkstraGraphEx.csv")
mg = create_graph(arr)

## Check values to make sure metagraph is correct
@test get_prop(mg,mg[arr[1][1],:name],mg[arr[1][2],:name],:weight) == 4
@test get_prop(mg,mg[arr[2][1],:name],mg[arr[2][2],:name],:weight) == 8
@test get_prop(mg,mg[arr[3][1],:name],mg[arr[3][2],:name],:weight) == 11
@test get_prop(mg,mg[arr[4][1],:name],mg[arr[4][2],:name],:weight) == 8
@test get_prop(mg,mg[arr[5][1],:name],mg[arr[5][2],:name],:weight) == 2
@test get_prop(mg,mg[arr[6][1],:name],mg[arr[6][2],:name],:weight) == 7
@test get_prop(mg,mg[arr[7][1],:name],mg[arr[7][2],:name],:weight) == 4
@test get_prop(mg,mg[arr[8][1],:name],mg[arr[8][2],:name],:weight) == 14
@test get_prop(mg,mg[arr[9][1],:name],mg[arr[9][2],:name],:weight) == 9
@test get_prop(mg,mg[arr[10][1],:name],mg[arr[10][2],:name],:weight) == 10
@test get_prop(mg,mg[arr[11][1],:name],mg[arr[11][2],:name],:weight) == 2
@test get_prop(mg,mg[arr[12][1],:name],mg[arr[12][2],:name],:weight) == 6
@test get_prop(mg,mg[arr[13][1],:name],mg[arr[13][2],:name],:weight) == 1
@test get_prop(mg,mg[arr[14][1],:name],mg[arr[14][2],:name],:weight) == 7

Using Dijsktra's Algorithm


Test Passed

In [8]:
arr2 = read_csv("BellmanFordEx.csv")
g = create_graph(arr2)[1]
dict_key = create_graph(arr2)[2]
for i in edges(g)
    println(i)
end
dict_key

Using Bellman-Ford's Algorithm


Using Bellman-Ford's Algorithm
Edge 1 => 2 with weight -1.0


Edge 1 => 3 with weight 4.0
Edge 2 => 3 with weight 3.0
Edge 2 => 4 with weight 2.0
Edge 2 => 5 with weight 2.0
Edge 4 => 2 with weight 1.0
Edge 4 => 3 with weight 5.0
Edge 5 => 4 with weight -3.0


Dict{Any, Any} with 5 entries:
  "B" => 2
  "A" => 1
  "C" => 3
  "D" => 4
  "E" => 5

# Dijsktra's Algorithm

Dijsktra's algorithm allows us to find the shortest path between any two vertices in a weighted graph, assuming that there exists a path between the two vertices.  This algorithm has broad applications such as finding the shortest path between the current location and the destination in apps like Google maps. The algorithm builds a set of vertices that have the minimum distance from the source, this algorithm is a greedy algorithm.
- A greedy algorithm is an approach to solving a problem by selecting the best possible option at the moment, given any conditions

The algorithm uses a priority queue to store the vertices that haven't been explored yet. The algorithm that I implemented below has a runtime of $\theta$(|V|^2) where |V| is the number of vertices
- It should be noted that by using a Fibonacci heap min priority queue, the runtime of Dijsktra's is cut down to $\theta$(|E| + |V| log(|V|))

Why does Dijsktra's algorithm work?
- Since the algorithm is greedy, it picks the vertex with the smallest path cost so far, any other path will have an equal or more cost. Thus finding the neighbors of vertex at the end of the smallest cost path will also be min cost.
- At each stage of the algorithm, it always checks that the vertex could lead to a shortest path, guaranteeing that all distances are minimum. Each iteration, you find the shortest path, or you can rule it out and move on to next vertex
- It exploits the fact that the shortest path between x and z includes some vertex y where there is a shortest path from x to y and another shortest path from y to z

Does Dijsktra's algorithm work with negative cost weights?
- No. Since Dijsktra's algorithm is a greedy algorithm, once a vertex is visited and shortest path is computed, it can't be recalculated even if there is a shorter path. This problem occurs only if there are negative weights in the graph. This is why Dijsktra's algorithm assumes that all edge weights are positive.

The pseudocode is as follows:
- For each vertex v in G:
	- Set distance of vertex v from source vertex as infinity
    - Set the previous vertex of v as NULL
    - Add v to a priority queue Q if v isn't the source vertex
- Set distance of source vertex to 0
- While Q isn't empty:
	- x = vertex in Q with smallest distance
    - For all unvisited neigbors y of x:
    	- temp_distance = distance(x) + edge_weight(x,y)
        - If temp_distance < distance(y):
        	- distance(y) = temp_distance
            - previous(y) = x
- Return distance[] and previous[]

In [9]:
## Input: The metagraph created earlier and a source vertex in the graph
## Output: Prints out the shortest path from source vertex to all other vertices and it's distance
function dijsktra_alg(meta_graph,src)
    ## Initiate distance, previous, and parent array
    ## dist_arr[i] holds the shortest distance from source vertex to vertex i
    ## prev_arr[i] will be true if vertex i is included in shortest path, or if i has been visited
    ## parent_arr[i] will hold index of the parent of i in shortest path (i.e. if vertex h is the second to last vertex in shortest path to i, then
    ## parent_arr[i] = h)
    dist_arr = Array{Float64}(undef,nv(meta_graph))
    prev_arr = Array{Bool}(undef,nv(meta_graph))
    parent_arr = Array{Float64}(undef,nv(meta_graph))
    ## Set the distance of all vertices to infinite and prev_arr[] to false
    for i in 1:nv(meta_graph)
        dist_arr[i] = Inf
        prev_arr[i] = false
    end
    ## Distance of source vertex from itself is always 0 and doesn't have any parents in shortest path
    dist_arr[meta_graph[src,:name]] = 0
    parent_arr[meta_graph[src,:name]] = -1
    ## Find shortest path for all vertices
    for i in 1:nv(meta_graph)
        ## Pick vertex with minimum distance and closest_vertex is always source vertex in first iteration
        closest_vertex = -1
        shortest_dist = Inf
        for j in 1:nv(meta_graph)
            if (prev_arr[j] == false) && (dist_arr[j] < shortest_dist)
                closest_vertex = j
                shortest_dist = dist_arr[j]
            end
        end
        ## Set closest_vertex as visited
        try
            prev_arr[closest_vertex] = true
        catch
            continue
        end
        ## Updates the distance of closest_vertex neighbors
        for k in 1:nv(meta_graph)
            edge_dist = 0
            try
                edge_dist = get_prop(meta_graph,closest_vertex,k,:weight)
            catch
                edge_dist = 0
            end
            if (edge_dist > 0) && ((shortest_dist + edge_dist) < dist_arr[k])
                parent_arr[k] = closest_vertex
                dist_arr[k] = shortest_dist + edge_dist
            end
        end
    end
    ## Prints out distance and shortest path from source vertex to all other vertices
    print_dijsktra(meta_graph,src,dist_arr,parent_arr)
end

## Helper function to print out distance from src to all other vertices and the path to get to that vertex
function print_dijsktra(meta_graph,src,dist_arr,parent_arr)
    for i in 1:nv(meta_graph)
        if i != meta_graph[src,:name]
            if dist_arr[i] == Inf
                println("Vertex: " * src * " -> " * meta_graph[i,:name] * "\tCould not find path")
            else
                print("Vertex: " * src * " -> " * meta_graph[i,:name] * "\t" * "Distance: " * string(dist_arr[i]) * "\tPath: "*src*" ")
                recursive_path(meta_graph,i,parent_arr)
                print("\n")
            end
        end
    end
end

## Helper function to print path with recursion
function recursive_path(meta_graph,src,parent_arr)
    if parent_arr[src] == -1
        return
    end
    parent_data = Int64(parent_arr[src])
    recursive_path(meta_graph,Int64(parent_arr[src]),parent_arr)
    print(string(meta_graph[src,:name])*" ")
end

recursive_path (generic function with 1 method)

## 1st Example

Before proceeding, try finding the shortest path from "7" to "4"

<img src="DijkstraAlgEx1.png"   width="816.725586px"  height="431.127747px"  style="object-fit:cover"/>

In [10]:
arr = read_csv("DijkstraGraphEx.csv")
mg = create_graph(arr)
## Vertex 99 and 21 are connected but in a different component, want to show that if there is no path to those two verices
dijsktra_alg(mg,"7")

Using Dijsktra's Algorithm


Vertex: 7 -> 0	Distance: 8.0	Path: 7 0 
Vertex: 7 -> 1	Distance: 11.0	Path: 7 1 
Vertex: 7 -> 2	Distance: 7.0	Path: 7 6 5 2 
Vertex: 7 -> 8	Distance: 7.0	Path: 7 8 
Vertex: 7 -> 3	Distance: 14.0	Path: 7 6 5 2 3 
Vertex: 7 -> 5	Distance: 3.0	Path: 7 6 5 
Vertex: 7 -> 4	Distance: 13.0	Path: 7 6 5 4 
Vertex: 7 -> 6	Distance: 1.0	Path: 7 6 
Vertex: 7 -> 99	Could not find path
Vertex: 7 -> 21	Could not find path


In [11]:
dijsktra_alg(mg,"8")

Vertex: 8 -> 0	Distance: 14.0	Path: 8 2 1 0 
Vertex: 8 -> 1	Distance: 10.0	Path: 8 2 1 
Vertex: 8 -> 7	Distance: 7.0	Path: 8 7 
Vertex: 8 -> 2	Distance: 2.0	Path: 8 2 
Vertex: 8 -> 3	Distance: 9.0	Path: 8 2 3 
Vertex: 8 -> 5	Distance: 6.0	Path: 8 2 5 
Vertex: 8 -> 4	Distance: 16.0	Path: 8 2 5 4 
Vertex: 8 -> 6	Distance: 6.0	Path: 8 6 
Vertex: 8 -> 99	Could not find path
Vertex: 8 -> 21	Could not find path


## 2nd Example

Try finding shortest path from "0" to "6"

<img src="DijkstraAlgEx2.png"   width="816.725586px"  height="431.127747px"  style="object-fit:cover"/>

In [12]:
arr2 = read_csv("DijkstraGraphEx2.csv")
mg2 = create_graph(arr2)
## Vertex 99 and 21 are connected but in a different component, want to show that if there is no path to those two verices
dijsktra_alg(mg2,"0")

Using Dijsktra's Algorithm
Vertex: 0 -> 1	Distance: 2.0	Path: 0 1 
Vertex: 0 -> 2	Distance: 6.0	Path: 0 2 
Vertex: 0 -> 3	Distance: 7.0	Path: 0 1 3 
Vertex: 0 -> 5	Distance: 22.0	Path: 0 1 3 5 
Vertex: 0 -> 4	Distance: 17.0	Path: 0 1 3 4 
Vertex: 0 -> 6	Distance: 19.0	Path: 0 1 3 4 6 


In [13]:
dijsktra_alg(mg2,"3")

Vertex: 3 -> 0	Distance: 7.0	Path: 3 1 0 
Vertex: 3 -> 1	Distance: 5.0	Path: 3 1 
Vertex: 3 -> 2	Distance: 8.0	Path: 3 2 
Vertex: 3 -> 5	Distance: 15.0	Path: 3 5 
Vertex: 3 -> 4	Distance: 10.0	Path: 3 4 
Vertex: 3 -> 6	Distance: 12.0	Path: 3 4 6 


## Bellman-Ford Algorithm

Bellman-Ford Algorithm is similar to Dijkstra's algorithm in that it is an algorithm that tries to find the shortest path between a source vertex and any other vertex reachable from that source vertex. The biggest difference between the two algorithms is that the Bellman-Ford algorithm will work on a graph with negative weights. However, the draw back is that Dijkstra's algorithm has a faster runtime compared to the Bellman-Ford algorithm
- Bellman-Ford algorithm has a runtime of $\theta$(|V|*|E|) where |V| is the number of vertices and |E| is the number of edges
- Remember that Dijkstra's runtime is $\theta$(|E| + |V| log(|V|))

Why does Bellman-Ford Algorithm work?
- Unlike Dijkstra's algorithm which is a greedy algorithm, Bellman-Ford algorithm is a dynamic programming algorithm
- The length of shortest path can be at most n-1 edges. The first iteration of the algorithm builds a possible path between two vertices with one edge. Each iteration tries to find a better path by adding one more edge. (i.e. on the i-th iteration it tries to find lowest cost path of i edges from source to each vertex)

Exceptions:
- The algorithm will not work with a graph that has a negative weight cycle
	- A negative weight cycle is a cycle where the sum of the edge weights are negative
    - It doesn't work because the algorithm can infinitely compute the shortest path by going through the negative weight cycle
- The algorithm will not work on an undirected graph
	- You can think of an undirected edge as a two way directed edge with one going from vertex u to v and another going from v to u
    - If there is a negative undirected edge weight, this is basically a negative weight cycle because the algorithm can keep going from u to v and vice versa

The psuedocode is as follows:
- For each vertex v in G
	- Set distance of vertex v as infinite from source vertex
    - Set previous vertex of v as NULL
- Set distance of source vertex to 0
- For each vertex v in G
	- For each edge (u,v) in G
    	- Set temp_distance to distance(u) + edge_weight(u,v)
        - If temp_distance < distance(v), set distance(v) to temp_distance and previous vertex of v to u
- For each edge (u,v) in G
	- If distance(u) + edge_weight(u,v) < distance(v), then there exists a negative weight cycle and return error
- Return distance[] and previous[]

In [14]:
## Input: The weighted directed graph created earlier and a source vertex in the graph (note: source vertex is the name of vertex, not the index of it)
## Output: Prints out the shortest path from source vertex to all other vertices and it's distance
function bellman_ford_alg(g,src,dict_key)
    ## Get index of source vertex
    src_vertex = dict_key[src]
    ## Initiate distance and predecessor array
    ## dist_arr[i] holds the shortest distance from source vertex to vertex i
    ## predecessor[i] holds index of previous vertex for shortest path from source vertex to vertex i
    dist_arr = Array{Float64}(undef,nv(g))
    predecessor_arr = Array{Int64}(undef,nv(g))
    ## Set distance of all vertices from source vertex to infinite
    for i in 1:nv(g)
        dist_arr[i] = Inf
    end
    ## Set distance of source vertex is 0 and predecssor to -1 since there is no path from source to source
    dist_arr[src_vertex] = 0
    predecessor_arr[src_vertex] = -1
    ## Relax all edges |V|-1 times. Shortest path can have at most |V|-1 edges
    for j in 1:nv(g)
        for k in edges(g)
            u = Graphs.src(k)
            v = Graphs.dst(k)
            edge_weight = get_weight(g,u,v)
            if (dist_arr[u] != Inf) && ((dist_arr[u] + edge_weight) < dist_arr[v])
                dist_arr[v] = dist_arr[u] + edge_weight
                predecessor_arr[v] = u
            end
        end
    end
    ## Loops one more time and if it finds a shorter path, then there is a negative cycle since the above step guarantees a shortest path
    for i in edges(g)
        x = Graphs.src(i)
        y = Graphs.dst(i)
        weight = get_weight(g,x,y)
        if (dist_arr[x] != Inf) && ((dist_arr[x] + weight) < dist_arr[y])
            return "Negative Weight Cycle Detected"
        end
    end
    ## Prints out shortest path to all vertices and distance
    print_dist(g,dist_arr,dict_key,predecessor_arr,src_vertex)
end

## Helper function to print out the shortest path and distance
function print_dist(g,dist_arr,dict_key,predecessor_arr,src)
    println("Vertex distance from source")
    for i in 1:nv(g)
        if i != src
            if dist_arr[i] == Inf
                println("Vertex: " * find_key(src,dict_key) * " -> " * find_key(i,dict_key) * "\tCould not find path")
            else
                print("Vertex: " * find_key(src,dict_key) * " -> " * find_key(i,dict_key) * "\tDistance: "* string(dist_arr[i])*"\tPath: ")
                print_path(g,dict_key,i,predecessor_arr)
                print("\n")
            end
        end
    end
end

## Helper function to print the path from source to all other vertices
function print_path(g,dict_key,target,predecessor_arr)
    curr = target
    path = []
    while curr != -1
        insert!(path,1,curr)
        curr = predecessor_arr[curr]
    end
    for i in path
        print(find_key(i,dict_key)*" ")
    end
end

## Helper function to find name of vertex given it's index
function find_key(val,dict_key)
    for k in keys(dict_key)
        if dict_key[k] == val
            return k
        end
    end
end

find_key (generic function with 1 method)

## 1st Example
Try finding shortest path from "A" to "E"

<img src="BellmanFordAlgEx1.png"   width="816.725586px"  height="600px"  style="object-fit:cover"/>

In [15]:
arr3 = read_csv("BellmanFordEx.csv")
g = create_graph(arr3)[1]
dict_key = create_graph(arr3)[2]
bellman_ford_alg(g,"D",dict_key)

Using Bellman-Ford's Algorithm
Using Bellman-Ford's Algorithm


Vertex distance from source
Vertex: D -> A	Could not find path
Vertex: D -> B	Distance: 1.0	Path: D B 
Vertex: D -> C	Distance: 4.0	Path: D B C 
Vertex: D -> E	Distance: 3.0	Path: D B E 


In [16]:
bellman_ford_alg(g,"A",dict_key)

Vertex distance from source
Vertex: A -> B	Distance: -1.0	Path: A B 
Vertex: A -> C	Distance: 2.0	Path: A B C 
Vertex: A -> D	Distance: -2.0	Path: A B E D 
Vertex: A -> E	Distance: 1.0	Path: A B E 


## 2nd Example
Try finding shortest path from "0" to "6"

<img src="BellmanFordAlgEx2.png"   width="816.725586px"  height="600px"  style="object-fit:cover"/>

In [17]:
arr4 = read_csv("BellmanFordEx2.csv")
g2 = create_graph(arr4)[1]
dict_key2 = create_graph(arr4)[2]
bellman_ford_alg(g2,"0",dict_key2)

Using Bellman-Ford's Algorithm
Using Bellman-Ford's Algorithm
Vertex distance from source
Vertex: 0 -> 4	Distance: 3.0	Path: 0 2 4 
Vertex: 0 -> 2	Distance: 2.0	Path: 0 2 
Vertex: 0 -> 1	Distance: 1.0	Path: 0 2 4 6 1 
Vertex: 0 -> 6	Distance: 6.0	Path: 0 2 4 6 


In [18]:
bellman_ford_alg(g2,"2",dict_key2)

Vertex distance from source
Vertex: 2 -> 0	Could not find path
Vertex: 2 -> 4	Distance: 1.0	Path: 2 4 
Vertex: 2 -> 1	Distance: -1.0	Path: 2 4 6 1 
Vertex: 2 -> 6	Distance: 4.0	Path: 2 4 6 


## Conclusion and Applications
As mentioned above, finding shortest path in a graph is applicable to a maps app like Google Maps, but there are a lot of other real life applications that can be used. For example, this can be used for social networking apps, telephone networking, and IP routing to find open shortest path. It is clear that graphs can be used to represent a lot of real life situations and finding shortest path between two vertices is fundamental to graph theory since the goal is to optimize applications.